In [1]:
import pandas as pd
import numpy as np

In [ ]:
results = pd.read_csv('result_portfolio.csv')
results_rand = pd.read_csv('result_portfolio_rand.csv')
import pandas as pd



In [8]:
results.columns

Index(['Date', 'NVDA', 'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG', 'META', 'TSLA',
       'AVGO',
       ...
       'MRNA_min', 'MRNA_max', 'DLTR_min', 'DLTR_max', 'WBA_min', 'WBA_max',
       'portfolio_max', 'portfolio_min', 'portfolio_buy', 'portfolio_curr'],
      dtype='object', length=297)

In [11]:
def filter_dataframe(df, columns):
    """
    Filters the DataFrame to include only the specified columns.

    Parameters:
    df (pd.DataFrame): The original DataFrame.
    columns (list): List of columns to include in the filtered DataFrame.

    Returns:
    pd.DataFrame: The filtered DataFrame.
    """
    return df[columns]




In [12]:
result = filter_dataframe(results, ['Date','portfolio_max', 'portfolio_min', 'portfolio_buy', 'portfolio_curr'])


result_rand = filter_dataframe(results_rand, ['Date','portfolio_max', 'portfolio_min', 'portfolio_buy', 'portfolio_curr'])

    

In [13]:
result

,Date,portfolio_max,portfolio_min,portfolio_buy,portfolio_curr
0,2023-01-03,33154.575195,19898.487549,19898.487549,32757.373047
1,2023-01-04,28593.326466,19384.669920,19939.178534,27342.463545
2,2023-01-05,31360.301196,19722.273417,19961.705865,30904.272795
3,2023-01-06,32314.643247,19770.075859,19940.196709,31997.908267
4,2023-01-09,28996.028625,18269.774780,19978.864563,27555.398651
...,...,...,...,...,...
467,2024-11-11,NaN,NaN,19928.339647,NaN
468,2024-11-12,NaN,NaN,19903.289836,NaN
469,2024-11-13,NaN,NaN,19995.299902,NaN
470,2024-11-14,NaN,NaN,19980.270325,NaN


In [17]:
import pandas as pd
import numpy as np
from typing import List, Tuple

def analyze_sequential_trades(df: pd.DataFrame, 
                          start_date: str,
                          initial_capital: float = 20000,
                          n_sequences: int = 4,
                          period: int = 60) -> Tuple[float, float, float, float]:
  """
  Analyze sequential trades using preprocessed max/min values
  """
  
  # Initialize variables
  current_capital = initial_capital
  start_idx = df[df['Date'] == start_date].index[0]
  trade_results = []
  
  # Execute sequential trades
  for seq in range(n_sequences):
      if start_idx >= len(df):
          break
          
      # Get row for this trade
      trade_row = df.iloc[start_idx]
      
      # Standardize based on current capital
      std_factor = current_capital / trade_row['portfolio_buy']
      
      # Calculate standardized values
      end_value = trade_row['portfolio_curr'] * std_factor
      period_max = trade_row['portfolio_max'] * std_factor
      period_min = trade_row['portfolio_min'] * std_factor
      
      # Calculate period drawdown
      period_drawdown = (period_max - period_min) / period_max if not np.isnan(period_max) else 0
      
      trade_return = (end_value - current_capital) / current_capital
      
      trade_results.append({
          'sequence': seq + 1,
          'start_date': trade_row['Date'],
          'start_capital': current_capital,
          'end_capital': end_value,
          'period_max': period_max,
          'period_min': period_min,
          'period_drawdown': period_drawdown,
          'return': trade_return
      })
      
      # Update for next sequence
      current_capital = end_value
      start_idx += period
  
  # Calculate overall metrics
  initial_value = initial_capital
  final_value = trade_results[-1]['end_capital']
  total_return = (final_value - initial_value) / initial_value
  
  # Calculate max drawdown across all sequences
  max_drawdown = max(trade['period_drawdown'] for trade in trade_results)
  
  # Calculate CAGR
  total_days = len(trade_results) * period
  cagr = (final_value / initial_value) ** (252 / total_days) - 1
  
  # Calculate Calmar ratio
  calmar_ratio = cagr / max_drawdown if max_drawdown != 0 else np.inf
  
#   # Print detailed results
#   print("\nDetailed Trade Results:")
#   for trade in trade_results:
#       print(f"Sequence {trade['sequence']} (Start Date: {trade['start_date']}):")
#       print(f"Start Capital: ${trade['start_capital']:.2f}")
#       print(f"End Capital: ${trade['end_capital']:.2f}")
#       print(f"Period Max: ${trade['period_max']:.2f}")
#       print(f"Period Min: ${trade['period_min']:.2f}")
#       print(f"Period Drawdown: {trade['period_drawdown']*100:.2f}%")
#       print(f"Return: {trade['return']*100:.2f}%\n")
  
  print(f"Overall Results:")
  print(f"Initial Capital: ${initial_value:.2f}")
  print(f"Final Capital: ${final_value:.2f}")
  print(f"Total Return: {total_return*100:.2f}%")
  print(f"Max Drawdown: {max_drawdown*100:.2f}%")
  print(f"CAGR: {cagr*100:.2f}%")
  print(f"Calmar Ratio: {calmar_ratio:.2f}")
  
  return total_return, max_drawdown, cagr, calmar_ratio



In [18]:
results = analyze_sequential_trades(result, '2023-01-03', 20000, 4, 60)

Overall Results:
Initial Capital: $20000.00
Final Capital: $52839.61
Total Return: 164.20%
Max Drawdown: 39.98%
CAGR: 177.35%
Calmar Ratio: 4.44


In [21]:
import pandas as pd
df=result.copy()
# Define start dates from the image
start_dates = [
    '2023-01-03',
    '2023-02-15',
    '2023-03-30',
    '2023-05-12',
    '2023-06-27',
    '2023-08-09',
    '2023-09-21',
    '2023-11-02'
]

# Create empty list to store results
results = []

# Process each start date
for start_date in start_dates:
    # Run the analysis using the existing function
    total_return, max_drawdown, cagr, calmar = analyze_sequential_trades(df, start_date)
    
    # Get end date (start date + 4 * 60 trading days)
    start_idx = df[df['Date'] == start_date].index[0]
    end_idx = min(start_idx + (4 * 60), len(df) - 1)
    end_date = df.iloc[end_idx]['Date']
    
    # Store results
    results.append({
        'Start_Date': start_date,
        'End_Date': end_date,
        'Total_PnL_%': round(total_return * 100, 2),
        'CAGR_%': round(cagr * 100, 2),
        'Max_Drawdown_%': round(max_drawdown * 100, 2),
        'Calmar_Ratio': round(calmar, 2)
    })

# Create DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('portfolio_analysis_results.csv', index=False)

# Display results
print("\nResults Summary:")
print(results_df)
print("\nResults saved to portfolio_analysis_results.csv")

Overall Results:
Initial Capital: $20000.00
Final Capital: $52839.61
Total Return: 164.20%
Max Drawdown: 39.98%
CAGR: 177.35%
Calmar Ratio: 4.44
Overall Results:
Initial Capital: $20000.00
Final Capital: $35014.28
Total Return: 75.07%
Max Drawdown: 28.55%
CAGR: 80.04%
Calmar Ratio: 2.80
Overall Results:
Initial Capital: $20000.00
Final Capital: $32334.48
Total Return: 61.67%
Max Drawdown: 28.08%
CAGR: 65.60%
Calmar Ratio: 2.34
Overall Results:
Initial Capital: $20000.00
Final Capital: $32802.90
Total Return: 64.01%
Max Drawdown: 31.20%
CAGR: 68.12%
Calmar Ratio: 2.18
Overall Results:
Initial Capital: $20000.00
Final Capital: $32582.26
Total Return: 62.91%
Max Drawdown: 36.61%
CAGR: 66.94%
Calmar Ratio: 1.83
Overall Results:
Initial Capital: $20000.00
Final Capital: $32734.16
Total Return: 63.67%
Max Drawdown: 31.20%
CAGR: 67.75%
Calmar Ratio: 2.17
Overall Results:
Initial Capital: $20000.00
Final Capital: $24232.99
Total Return: 21.16%
Max Drawdown: 36.61%
CAGR: 22.33%
Calmar Ratio: 0.

In [22]:
import pandas as pd
df=result_rand.copy()
# Define start dates from the image
start_dates = [
    '2023-01-03',
    '2023-02-15',
    '2023-03-30',
    '2023-05-12',
    '2023-06-27',
    '2023-08-09',
    '2023-09-21',
    '2023-11-02'
]

# Create empty list to store results
results = []

# Process each start date
for start_date in start_dates:
    # Run the analysis using the existing function
    total_return, max_drawdown, cagr, calmar = analyze_sequential_trades(df, start_date)
    
    # Get end date (start date + 4 * 60 trading days)
    start_idx = df[df['Date'] == start_date].index[0]
    end_idx = min(start_idx + (4 * 60), len(df) - 1)
    end_date = df.iloc[end_idx]['Date']
    
    # Store results
    results.append({
        'Start_Date': start_date,
        'End_Date': end_date,
        'Total_PnL_%': round(total_return * 100, 2),
        'CAGR_%': round(cagr * 100, 2),
        'Max_Drawdown_%': round(max_drawdown * 100, 2),
        'Calmar_Ratio': round(calmar, 2)
    })

# Create DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('random_results.csv', index=False)

# Display results
print("\nResults Summary:")
print(results_df)
print("\nResults saved to random_results.csv")

Overall Results:
Initial Capital: $20000.00
Final Capital: $37294.83
Total Return: 86.47%
Max Drawdown: 36.21%
CAGR: 92.38%
Calmar Ratio: 2.55
Overall Results:
Initial Capital: $20000.00
Final Capital: $36498.43
Total Return: 82.49%
Max Drawdown: 31.29%
CAGR: 88.06%
Calmar Ratio: 2.81
Overall Results:
Initial Capital: $20000.00
Final Capital: $28941.75
Total Return: 44.71%
Max Drawdown: 25.42%
CAGR: 47.41%
Calmar Ratio: 1.86
Overall Results:
Initial Capital: $20000.00
Final Capital: $25464.01
Total Return: 27.32%
Max Drawdown: 31.29%
CAGR: 28.87%
Calmar Ratio: 0.92
Overall Results:
Initial Capital: $20000.00
Final Capital: $25270.43
Total Return: 26.35%
Max Drawdown: 25.42%
CAGR: 27.84%
Calmar Ratio: 1.09
Overall Results:
Initial Capital: $20000.00
Final Capital: $26919.18
Total Return: 34.60%
Max Drawdown: 37.84%
CAGR: 36.61%
Calmar Ratio: 0.97
Overall Results:
Initial Capital: $20000.00
Final Capital: $24330.02
Total Return: 21.65%
Max Drawdown: 27.05%
CAGR: 22.85%
Calmar Ratio: 0.84